<table style="width: 100%; border-style: none">
 <tbody>
  <tr style="border-style: none">
   <td style="border-style: none; width: 1%; text-align: left; font-size: 16px">Institute for Biomedical Imaging<br />Hamburg University of Technology</td>
   <td style="border-style: none; width: 1%; font-size: 16px">&nbsp;</td>
   <td style="border-style: none; width: 1%; text-align: right; font-size: 16px">Dr. Martin Möddel<br />Marija Boberg<br />Mirco Grosser</td>
  </tr>
 </tbody>
</table>
<hr>
<h1 style="font-weight:bold; text-align: center; margin: 0px; padding:0px;">Computer Graphics</h1>
<h1 style="font-weight:bold; text-align: center; margin: 0px; padding:0px;">Exercise 4 - Rasterization</h1>
<h3 style="font-weight:bold; text-align: center; margin: 0px; padding:0px; margin-bottom: 20px;">summer term 2023</h3>
<hr>

* 📅 Due date: 30.05.2022, 11 a.m.
* You can earn 1 point for each task (5 points in total).
* 💣 The last two tasks are small coding challenges for those interested (1 bonus point each).

### 1. Rendering a Sphere
While the last exercise primarily dealt with simple 2-dimensional objects, we are now going to render more complex 3-dimensional objects. In order to render such objects we need a way to load the vertices and the indices making up each face from files.
For this purpose consider the following code, which extracts the given information from obj-files.

With this code at hand, load the sphere contained in the file `sphere.obj` and render it. This involves the following steps:
* store the vertices in a VBO and create an EBO for the indices making up the faces. 
* link the VBO to the current VAO.
* implement simple shaders such that each fragment is renderd in a fixed color that you like

For illustration also draw the wireframe of the primitives. This can be achieved by setting the plotting mode to wireframe mode, using `glPolygonMode(GL_FRONT_AND_BACK, GL_LINE)`.


*Remarks:*
* For this exercise, you need to add the packages `FileIO`, `ImageMagick`, and `ImageIO`.
* The file `utils.jl` contains the functions `initWindow`, `key_callback` and `compileShaderProgram`, which were implemented in the last exercise.

In [1]:
function load_obj(filename::String, scale=0.7)
    vertices = Array{Float32}(undef,0)
    indices = Array{UInt32}(undef,0)

    # read .obj file line for line
    for line in readlines(filename)
        if line[1:2] == "v "
            # read the x, y and z position of each vertex
            strvals = split(line,' ')[2:4]
            push!(vertices,map(x->parse(Float32,x),strvals)...)
            vertices[end] *= -1
        elseif line[1:2] == "f "
            # read the triple of references
            strvals = split(line,[' '])[2:4]
            push!(indices,map(x->parse(UInt16,x)-1,strvals)...)
        end
    end

    # scale down object in object coordinate sytem
    map!(x->scale*x,vertices,vertices)
    return vertices, indices
end

v, i = load_obj("./objects/sphere.obj")

(Float32[0.0, 0.31304953, -0.62609905, -0.25997347, 0.29065916, -0.5813183, -0.12998672, 0.089284554, -0.6820057, 0.12998672  …  0.46769315, -0.26078948, -0.33109915, 0.55889374, -0.34986234, -0.08524055, 0.6002753, -0.41123065, 0.12993482, 0.55136764], UInt32[0x00000000, 0x00000001, 0x00000002, 0x00000000, 0x00000002, 0x00000003, 0x00000000, 0x00000003, 0x00000004, 0x00000000  …  0x00000076, 0x00000070, 0x0000006f, 0x00000076, 0x0000006f, 0x0000006e, 0x00000076, 0x00000076, 0x00000075, 0x0000006e])

In [4]:
using GLFW
using ModernGL
using FileIO
using ImageMagick
using ImageIO 
# just imported everything once

include("utils.jl")

# create window and openGL context from utils.jl
window = initWindow(title = "Sphere", callbackFunc = key_callback)

# Load the vertices and indices of the sphere from the sphere.obj file using function above
vertices, indices = load_obj("./objects/sphere.obj")


# Create a Vertex Buffer Object (VBO) and an Element Buffer Object (EBO) to store the vertex and index data
vbo = Ref(GLuint(0))
ebo = Ref(GLuint(0))

glGenBuffers(1, vbo)
glGenBuffers(1, ebo)

glBindBuffer(GL_ARRAY_BUFFER, vbo[])
glBufferData(GL_ARRAY_BUFFER, sizeof(vertices), vertices, GL_STATIC_DRAW)

glBindBuffer(GL_ELEMENT_ARRAY_BUFFER, ebo[])
glBufferData(GL_ELEMENT_ARRAY_BUFFER, sizeof(indices), indices, GL_STATIC_DRAW)

# Vertex Array Object (VAO) and link the VBO and EBO to the VAO.
vao = Ref(GLuint(0))
glGenVertexArrays(1, vao)
glBindVertexArray(vao[])

glBindBuffer(GL_ARRAY_BUFFER, vbo[])
glVertexAttribPointer(0, 3, GL_FLOAT, GL_FALSE, 0, C_NULL)
glEnableVertexAttribArray(0)

glBindBuffer(GL_ELEMENT_ARRAY_BUFFER, ebo[])

# Compile the vertex and fragment shaders using the compileShaderProgram function from utils.jl
vertex_source = """
#version 330 core

layout (location = 0) in vec3 position;

void main()
{
    gl_Position = vec4(position, 1.0);
}
"""

fragment_source = """
#version 330 core

out vec4 FragColor;

void main()
{
    FragColor = vec4(0.5, 0.0, 0.5, 1.0);  // Fixed color (violet)
}
"""

shader_program = compileShaderProgram(vertex_source, fragment_source)

# Set the polygon mode to wireframe mode using glPolygonMode to enable drawing the wireframe
glPolygonMode(GL_FRONT_AND_BACK, GL_LINE)

# Inside the rendering loop, bind the shader program, bind the VAO, and call glDrawElements to render the sphere.
while !GLFW.WindowShouldClose(window)
    glClear(GL_COLOR_BUFFER_BIT)
    
    glUseProgram(shader_program)
    glBindVertexArray(vao[])
    glDrawElements(GL_TRIANGLES, length(indices), GL_UNSIGNED_INT, C_NULL)
    
    GLFW.SwapBuffers(window)
    GLFW.PollEvents()
end

GLFW.DestroyWindow(window)

Dict{GLFW.Window, Ref{Vector{Union{Nothing, Function}}}}()

### 2. Coloring the Sphere

The following code generates colors for each vertex of the sphere. This color is determined solely by the latitude of the given vertex. Moreover, we provide a method, which shuffles the order in which the faces are stored.

Extend your previous code to render the colored sphere. This can be done by following the following steps:
* store the vertices and colors in separate VBOs.
* link both VBOs to the current VAO.
* modify the vertex shader such that the vertex color is an additional input. The vertex should then be assigned to an additional output variable.
* modify the fragment shader, such that the fragment color is an additional input. Assign this input as the shaders output.

First render the scene without shuffling the faces. Then shuffle the faces and render the scene again. What do you observe? How do you explain your observations? 

In [4]:
using Random

# load vertices and define their colors 
vertices, indices = load_obj("objects/sphere.obj", 0.7)
numVertices = div(length(vertices),3)
vertexColor = zeros(Float32,3*numVertices)
for i=1:numVertices
    x,y,z = vertices[3*(i-1)+1:3*i]
    θ = acos(z/sqrt(x^2+y^2+z^2))
    vertexColor[3*(i-1)+1:3*i] .= Float32.([0.176 - θ/pi*0.176, 0.776 - θ/pi*0.776, 0.839 - θ/pi*0.239])
end

# permute faces
function shuffleFaces!(indices::Vector{UInt32})
    numFaces = div(length(indices),3)
    p = shuffle(collect(1:numFaces))
    # indices = reshape(indices,3,numFaces)
    indices .= vec(reshape(indices,3,numFaces)[:,p])
end

shuffleFaces!(indices);

In [4]:
import GLFW
using ModernGL
using Random

include("utils.jl")

# Create a window and a corresponding OpenGL context
window = initWindow(title="Rendering Colored Sphere", callbackFunc = key_callback)

# Compile shader program
vertex_source = """
#version 330 core
layout (location = 0) in vec3 position;
layout (location = 1) in vec3 color;

out vec3 fragColor;

void main()
{
    gl_Position = vec4(position, 1.0);
    fragColor = color;
}
"""

fragment_source = """
#version 330 core
in vec3 fragColor;
out vec4 FragColor;

void main()
{
    FragColor = vec4(fragColor, 1.0);
}
"""

shader_program = compileShaderProgram(vertex_source, fragment_source)

# Load vertices and define their colors
# vertices, indices = load_obj("objects/sphere.obj")
vertices, indices = load_obj("objects/sphere.obj", 0.7)
numVertices = div(length(vertices), 3)
vertexColor = zeros(Float32, 3*numVertices)

for i = 1:numVertices
    x, y, z = vertices[3*(i-1)+1:3*i]
    θ = acos(z / sqrt(x^2 + y^2 + z^2))
    vertexColor[3*(i-1)+1:3*i] .= Float32[0.176f0 - θ / pi * 0.176f0, 0.776f0 - θ / pi * 0.776f0, 0.839f0 - θ / pi * 0.239f0]
end

# # permute faces
# function shuffleFaces!(indices::Vector{UInt32})
#     numFaces = div(length(indices),3)
#     p = shuffle(collect(1:numFaces))
#     # indices = reshape(indices,3,numFaces)
#     indices .= vec(reshape(indices,3,numFaces)[:,p])
# end

# # Shuffle faces meaning, shuffling faces?
# shuffleFaces!(indices)

# Vertex Buffer Objects (VBOs)
vboVertices = Ref(GLuint(0))
glGenBuffers(1, vboVertices)
glBindBuffer(GL_ARRAY_BUFFER, vboVertices[])
glBufferData(GL_ARRAY_BUFFER, sizeof(vertices), vertices, GL_STATIC_DRAW)

vboColors = Ref(GLuint(0))
glGenBuffers(1, vboColors)
glBindBuffer(GL_ARRAY_BUFFER, vboColors[])
glBufferData(GL_ARRAY_BUFFER, sizeof(vertexColor), vertexColor, GL_STATIC_DRAW)

# Vertex Array Object (VAO)
vao = Ref(GLuint(0))
glGenVertexArrays(1, vao)
glBindVertexArray(vao[])

# Link vertex data to attributes
posAttribute = glGetAttribLocation(shader_program, "position")
glBindBuffer(GL_ARRAY_BUFFER, vboVertices[])
glVertexAttribPointer(posAttribute, 3, GL_FLOAT, GL_FALSE, 0, C_NULL)
glEnableVertexAttribArray(posAttribute)

colorAttribute = glGetAttribLocation(shader_program, "color")
glBindBuffer(GL_ARRAY_BUFFER, vboColors[])
glVertexAttribPointer(colorAttribute, 3, GL_FLOAT, GL_FALSE, 0, C_NULL)
glEnableVertexAttribArray(colorAttribute)

# Set wireframe mode
glPolygonMode(GL_FRONT_AND_BACK, GL_LINE)

# Main rendering loop
while !GLFW.WindowShouldClose(window)
    # Clear the color buffer
    glClear(GL_COLOR_BUFFER_BIT)
    
    # Use the shader program
    glUseProgram(shader_program)
    
    # Bind the vertex array object
    glBindVertexArray(vao[])
    
    # Draw the sphere
    glDrawElements(GL_TRIANGLES, length(indices), GL_UNSIGNED_INT, indices)
    
    # Swap front and back buffers
    GLFW.SwapBuffers(window)
    
    # Poll for and process events
    GLFW.PollEvents()
end

GLFW.DestroyWindow(window)

Dict{GLFW.Window, Ref{Vector{Union{Nothing, Function}}}}()

I don't see any difference with shuffled faces.

 ### 3. Depth-Testing
 
Modify your code such that depth-testing is enabled. This can be achieved with the code `glEnable(GL_DEPTH_TEST)`.
Furthermore, depth-testing requires you to clear the depth-buffer-bit in each rendering cycle. This can be done using `glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)`.

Use your code to render the sphere, after shuffling its faces. OpenGL supports multiple depth functions which can be set using the method `glDepthFunc(func)`. Valid depthfunctions include `GL_NEVER`, `GL_LESS`, `GL_EQUAL` and `GL_GREATER`, among others. 
Play around with the different depth-functions until the sphere looks like one expects it to.

### 4. Texture Mapping

A more practical way of coloring objects is texture mapping.

Let us turn our sphere into the saturn. For this purpose we use the texture map `tex/saturn.png`. The 2-dimensional texture coordinates of the vertices are given in the following cell.

Extend the code, such that the fragment shader outputs the corresponding color in the texture map. To achieve this, the following steps need to be taken:
* Modify the vertex shader such that the texture coordinate is passed through, analogously to the color in exercise 1.
* Add a `uniform sampler2d satTex` to the fragment shader. Have the fragment shader return the RGBA-values contained in the corresponding location of the texture. In order to sample the texture at a coordtinate `ftCoordinate`, you can use the function call `texture(satTex, ftCoordinate)`.
* To store the texture coordinates you can use a VBO as in the previous task. 
* Setting up the texture itself works analously to setting up a VBO. This requires you to use the functions `glGenTextures` and `glBindTexture`.
* Finally, one should specify the wrapping behaviour of the texture map for the s- and t-coordinates. This can be done by setting the parameters `GL_TEXTURE_WRAP_S` and `GL_TEXTURE_WRAP_T` with the functions  `glTexParameteri`.
* Specifying the texture image can be done using the function `glTexImage2D`.
* Set the parameters `GL_TEXTURE_MIN_FILTER` and `GL_TEXTURE_MAG_FILTER`.

What is the significance of the parameters `GL_TEXTURE_MIN_FILTER` and `GL_TEXTURE_MAG_FILTER`? What do you observe, when changing the functions used?

In [7]:
planetVertices, planetIndices = load_obj("objects/sphere.obj",0.4)

# texture coordinatest
planetUV = zeros(Float32, 2*div(length(planetVertices),3))
r = maximum(planetVertices)
for i=0:div(length(planetVertices),3)-1
    xi,yi,zi = planetVertices[3*i+1:3*i+3]
    planetUV[2*i+2] = (atan(yi,xi)/pi+1)*0.5
    planetUV[2*i+1] = acos(yi/sqrt(xi^2+yi^2+zi^2))/pi
end

### 5. Lambert Shading

Render the previous scene with Lambert shading. This requires you to perform the following steps:
* Compute the normal vectors for each vertex. For this purpose, first compute the normal vectors for all faces. Then calculate the normal for a vertex by averaging the normal vectors of the faces which that vertex contributes to.
* Implement Lambert shading in the fragment shader. This requires you to pass the normal vectors through the vertex shader. In order to have access to the position associated with each fragment, one can simply assign the vertex positions to an additional output variable in the vertex shader. During rasterization, a corresponding position variable will be attached to each fragment. This variable then becomes an input to the fragment shader and is thus available for lighting computations.
* Build an additional VBO to store the normal vectors.

### B1. Instancing

Next, we are going to add an asteroid field to our scene. The latter will be formed by rendering the same rock object at different positions around the planet. 

The direct way to approach this is to have to have multiple calls to `glDrawElements` - namely one for each rock. Prior to each rendering call, a new shift-vector is specified via a uniform. This approach is not optimal, because each rendering call requires a significant amount of data-movement over the CPU-to-GPU bus. Thus, performance is not limited by the actual rendering of the vertices but rather by the data-transfer associated with each draw-command.

Instancing is a way to draw multiple instances of an object using a single-rendering call. This implies that all the data needed for the rendering needs to be passed to the GPU only once.

To render our planet with the asteroid field using instancing, we start by generating a new window. Then we load vertices and indices of the rock object. Additionally, we compute its normal vectors and we generate a randomized array of shifts to position the rocks around the planet.

In [11]:
# planet coordinates
planetVertices, planetIndices = load_obj("objects/sphere.obj",0.4)
planetUV = zeros(Float32, 2*div(length(planetVertices),3))
r = maximum(planetVertices)
for i=0:div(length(planetVertices),3)-1
    xi,yi,zi = planetVertices[3*i+1:3*i+3]
    planetUV[2*i+2] = (atan(yi,xi)/pi+1)*0.5
    planetUV[2*i+1] = acos(yi/sqrt(xi^2+yi^2+zi^2))/pi
end
planetNormals = normalVectors(planetVertices, planetIndices);

# rock
rockVertices, rockIndices = load_obj("objects/rock.obj",0.005)
rockNormals = normalVectors(rockVertices, rockIndices)

# translation vectors
numRocks = 10000
r0 = 0.6
trans = Matrix{Float32}(undef,3,numRocks)
# rotation around the x axis
Random.seed!(1234)
θ = deg2rad(30)
R = Float32[1 0 0; 0 cos(θ) sin(θ); 0 -sin(θ) cos(θ)]
for i=1:numRocks
    ϕ = 2*pi*rand()
    y = 0.1*(rand()-0.5)
    r = r0+0.2*rand()
    trans[:,i] = R*Float32.( [r*cos(ϕ), y, r*sin(ϕ)] )
end

trans = vec(trans);

UndefVarError: UndefVarError: normalVectors not defined

We need two shader programs. The first one is the shader program for the planet itself, which we have implemented in the previous exercise. 

Implement an additional set of shaders for the rocks. The vertex shader should have three inputs:
* the vertex position in "object coordinates"
* the normal vector
* a translation vector

In the vertex shader, the shifted vertex position should be assigned to `glPosition`. Moreover, it should output the shifted vertex position and the normal vector.

The fragment shader should implement Lambert shading. For simplicity we do not use a texture map for the rock. Instead all rock fragments should have the color (0.3,0.3,0.3,1.0) in RGBA coordinates.

Now render the complete scene: 

* To render the planet, use the same VAO, VBOs and EBO as in the previous task. 
* Create a second VAO (and associated VBOs and EBO) for the rocks. For this part you will need three VBOs - for the vertices, the normal vectors and the translation vectors. Make sure to tell OpenGL, that the shift vector should be updated everytime when we start to render a new instance. This can be achieved using the method `glVertexAttribDivisor(index, 1)`, where `index` refers to the index of the attribute (i.e. the shift vectors).
* For instanced rendering, the draw command `glDrawElements` needs to be replaced with `glDrawElementsInstanced`. Check the OpenGL documentation for details about the method's parameters. 
* Make sure to activate the correct VAO and the correct shader program before each draw command. This can be done using the methods `glBindVertexArray` and `glUseProgram`

### B2. Cube Maps
Cube maps are a combination of 6 2d texture maps. By combining these texture maps, one can assign a unique texture for all possible direction vectors. This yields a powerful technique, which can be used to generate texure-map based backgrounds for instance.

In order to add a background to our scene, we can use the cube map contained in the folder `tex/milkyway`. We then render an additional cube spanning the volume $[-1,1]^3$. Each point of the cube should be colored according to the corresponding value in the cube map. To generate the vertices and face indices of the cube execute the following code, which loads the vertex data of all the objects considered so far.

Note that you are not required to render the asteroid belt of taks B1 in this exercise. However, you are free to do so if you want.

In [14]:
######################
# vertices and indices
######################
# planet
planetVertices, planetIndices = load_obj("objects/sphere.obj",0.4)
planetNormals = normalVectors(planetVertices, planetIndices)
planetUV = zeros(Float32, 2*div(length(planetVertices),3))
r = maximum(planetVertices)
for i=0:div(length(planetVertices),3)-1
    xi,yi,zi = planetVertices[3*i+1:3*i+3]
    planetUV[2*i+2] = (atan(yi,xi)/pi+1)*0.5
    planetUV[2*i+1] = acos(yi/sqrt(xi^2+yi^2+zi^2))/pi
end

# skybox
# front face
v1 = Float32[-1.0,-1.0,1.0, 1.0,-1.0,1.0, 1.0, 1.0,1.0, -1.0, 1.0,1.0]
# back face
v2 = Float32[1.0,-1.0,-1.0, -1.0,-1.0,-1.0, -1.0, 1.0,-1.0, 1.0, 1.0,-1.0]
# right face
v3 = Float32[1.0,-1.0,1.0, 1.0,-1.0,-1.0, 1.0,1.0,-1.0, 1.0,1.0,1.0]
# left face
v4 = Float32[-1.0,-1.0,-1.0, -1.0,-1.0,1.0, -1.0,1.0,1.0, -1.0,1.0,-1.0]
# top face
v5 = Float32[-1.0,1.0,1.0, 1.0,1.0,1.0, 1.0,1.0,-1.0, -1.0,1.0,-1.0]
# bottom face
v6 = Float32[-1.0,-1.0,-1.0, 1.0,-1.0,-1.0, 1.0,-1.0,1.0, -1.0,-1.0,1.0]

# indices for triangles
skyboxIndices = UInt32[0,1,2,
                2,3,0,
                4,5,6,
                6,7,4,
                8,9,10,
                10,11,8,
                12,13,14,
                14,15,12,
                16,17,18,
                18,19,16,
                20,21,22,
                22,23,20]

skyboxVertices = vcat(v1,v2,v3,v4,v5,v6)

# rock
rockVertices, rockIndices = load_obj("objects/rock.obj",0.005)
rockNormals = normalVectors(rockVertices, rockIndices);

UndefVarError: UndefVarError: normalVectors not defined

Implement an additional set of shaders which we are going to use for the skybox. The vertex shader should assign the vertex position to the variable `gl_Position`. Moreover, the 3d-texture coordinates should be passed on to the fragment shader. The fragment shader should determine the output color by sampling the cube map. To do so, the cube map needs to be passed to the shader as a `uniform samplerCube skyTex`. The texture can then be sampled using `texture(skyTex, texCoords)`. 

Generate a window, compile the shader programs and set up the data structures (VAO,..,) for the rest of the scene (not including the skybox). For this purpose you can use the code you generated in the previous tasks.

Now, set up the data structures for the skybox. As was done before, the data associated needs to be stored in a separate VAO and corresponding VBOs and EBO.

To generate the corresponding texture and bind it to the texture target one can proceed like with a regular texture map, albeit with a different target
```
texture = Array{GLuint}(undef,1)
glGenTextures(1, texture)
glBindTexture(GL_TEXTURE_2D, texture[1])
```
In order to load the six textures, one needs to call `glTexImage2D` six times with the targets `GL_TEXTURE_CUBE_MAP_POSITIVE_X`, `GL_TEXTURE_CUBE_MAP_NEGATIVE_X` ,`GL_TEXTURE_CUBE_MAP_POSITIVE_Y`, `GL_TEXTURE_CUBE_MAP_NEGATIVE_Y`, `GL_TEXTURE_CUBE_MAP_POSITIVE_Z`, `GL_TEXTURE_CUBE_MAP_NEGATIVE_Z`. The textures in the folder have the format RGBA. So make sure to specify it correctly.

Analogously to regular texture maps, we need to specify their wrapping behaviour. However, in this case we need to call `glTexParameteri` for three parameters `GL_TEXTURE_WRAP_S`, `GL_TEXTURE_WRAP_T` and `GL_TEXTURE_WRAP_R`.

Finally, let us render the complete scene.
Before rendering the skybox, use the following code
```
glDepthMask(GL_FALSE)
glUseProgram(skybox_shader)
glBindVertexArray(skyboxVAO[1])
```
After rendering the skybox turn the DepthMask back on (`glDepthMask(GL_TRUE)`) and render the rest of the scene.